In [ ]:
import numpy as np
import gpxpy
import pyproj
import folium
from branca.colormap import linear
import fullcontrol as fc



In [ ]:
gpx_path = "example_rides/Lunch_Ride.gpx"

## Open a gpx file
lat_long_points= np.empty((0, 2))
elevation = np.empty((0, 1))

# Parsing a GPX file
with open(gpx_path, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            lat_long_points = np.vstack((lat_long_points, np.array([point.latitude, point.longitude])))
            elevation = np.append(elevation, point.elevation)

In [ ]:
## Calculate zone from lat/long

centre = np.min(lat_long_points, axis=0) + (np.ptp(lat_long_points, axis=0) / 2)

zone = int((centre[1] + 180) / 6) + 1

P = pyproj.Proj(proj='utm', zone=zone, ellps='WGS84', preserve_units=True)
G = pyproj.Geod(ellps='WGS84')

utm_points = np.empty((0, 2))
for point in lat_long_points:
    utm_points = np.vstack((utm_points, P(point[1], point[0])))

lat_long_centre = np.min(lat_long_points, axis=0) + (np.ptp(lat_long_points, axis=0) / 2)


In [ ]:
## Create a new list of points that contains one point for every 10m of the points (smoothing things out)
distance_sum = 0
point_couter = 0
utm_position_sum = np.array([0,0], dtype=float)
lat_long_position_sum = np.array([0,0], dtype=float)
last_pos = utm_points[0].astype(np.float64)

lat_long_points_smooth = np.array([lat_long_points[0]], dtype=float)
utm_points_smooth = np.array([utm_points[0]], dtype=float)
new_elevation = np.array([elevation[0]], dtype=float)

for i in range(1, len(utm_points)):
    distance = np.sqrt(np.sum((utm_points[i].astype(np.float64) - last_pos)**2))
    utm_position_sum += utm_points[i].astype(np.float64)  # Explicitly convert position_sum to float64
    lat_long_position_sum += lat_long_points[i].astype(np.float64)  # Explicitly convert position_sum to float64
    point_couter += 1
    if distance >= 25:
        utm_points_smooth = np.vstack((utm_points_smooth, utm_position_sum/point_couter))
        lat_long_points_smooth = np.vstack((lat_long_points_smooth, lat_long_position_sum/point_couter))
        point_couter = 0
        last_pos = utm_points[i].astype(np.float64)
        utm_position_sum = np.array([0,0], dtype=float)
        lat_long_position_sum = np.array([0,0], dtype=float)
        new_elevation = np.append(new_elevation, elevation[i])

print(len(utm_points_smooth), len(lat_long_points_smooth), len(new_elevation))



In [ ]:
## Create a colormap based on altitude values
altitude_colormap = linear.Spectral_11.scale(new_elevation.min(), new_elevation.max())

colours = list(map(altitude_colormap, new_elevation))

## Create a function to assign color based on altitude
def color_function(altitude):
    return altitude_colormap(altitude)

## Show map using utm coordinates
m = folium.Map(location=[lat_long_centre[0], lat_long_centre[1]], zoom_start=11)
folium.TileLayer('cartodbpositron').add_to(m)

## Add overlay of lines onto map with color based on altitude
for i in range(1, len(lat_long_points_smooth)):
    folium.PolyLine([lat_long_points_smooth[i-1], lat_long_points_smooth[i]], color=color_function(new_elevation[i-1]), weight=2.5).add_to(m)


## Add colorbar to the map
altitude_colormap.caption = 'Altitude'
m.add_child(altitude_colormap)

In [ ]:

gcode = []

layers = 15

smoothing = 33
wrap = smoothing//2

# apply some kind of smoothing to the altitude
smooth_elevation = np.concatenate((new_elevation[:-wrap], new_elevation, new_elevation[:wrap]))
smooth_elevation = np.convolve(smooth_elevation, np.ones(smoothing)/smoothing, mode='valid')

scaled_points = utm_points_smooth/130
scaled_points -= np.min(scaled_points, axis=0)
scaled_points += np.array([10,50])
# scaled_points += np.array([30,30])
elevation_factor = ((smooth_elevation-np.min(smooth_elevation))/(np.ptp(smooth_elevation))*0.75)+0.25

gcode.append(fc.ManualGcode(text='M601;'))

for i in range(layers):
    for j, point in enumerate(scaled_points):
        gcode.append(fc.ExtrusionGeometry(area_model='stadium', width = 1, height = 0.4*elevation_factor[j]))
        gcode.append(fc.Point(x=point[0], y=point[1],z=0.6+(0.4*i)*elevation_factor[j]))
    
gcode_controls = fc.GcodeControls(printer_name='generic', initialization_data={'print_speed': 20*60}) 
controls=fc.PlotControls(style='tube')

fc.transform(gcode, 'plot', controls)

In [ ]:
lines = fc.transform(gcode, 'gcode', gcode_controls)
with open('out.gcode', 'w') as f:
    f.write(lines)